#### make sure dataset is downloaded and extracted `@repoDirectory/data/raw/font-datas0et` -> this path should be at the repo root

In [6]:
#install all dependencies
%pip install -r "../../requirements.txt"

2318.90s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
